---
# LangChain SQL Microservice with Jupyter Notebook

## Introduction
This notebook specifically focuses on setting up the Chinook database and integrating LangChain for seamless data querying.

## Prerequisites
- Basic knowledge of Python and SQL.
- An environment for running Jupyter Notebooks (e.g., Google Colab, JupyterLab).

## Installation
Clone the repository to your local machine or open it in a Jupyter Notebook compatible environment:
```
git clone [repository-link]
```

## Setting Up
1. **Install Dependencies**: Run the following command in your notebook to install necessary packages.
   ```python
   !pip install langchain langchain-experimental openai
   ```
2. **Configure OpenAI API Key**: Replace `'your_api_key_here'` with your actual OpenAI API key.
   ```python
   import os
   os.environ['OPENAI_API_KEY'] = 'your_api_key_here'
   ```

## Usage
1. **Initialize LangChain with SQLDatabaseChain**:
   ```python
    from langchain.llms import OpenAI
    from langchain.utilities import SQLDatabase
    from langchain_experimental.sql import SQLDatabaseChain

    # Connect to the Chinook database
    db = SQLDatabase.from_uri("sqlite:///Chinook.db")
    llm = OpenAI(temperature=0, verbose=True)
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
   ```

2. **Querying the Database**: Use natural language prompts to query the database.
   ```python
    try:
        response = db_chain.run("How many employees are there?")
        print("Response:", response)
    except Exception as e:
        print("Error:", e)

   ```

## Deep Dive into SQL Queries
- Explore advanced SQL queries constructed from user questions using LangChain.
- Example code snippets are provided in the notebook.

## Enhancing the Microservice
- Customization of the database description.
- Implementation of error recovery with SQL agents.

## Testing
- Test cases are included to demonstrate the functionality of the microservice.
- Follow the interactive code cells for experimenting with the microservice.

## Contributing
Contributions to this project are welcome. Please adhere to the following guidelines:
1. Fork the repository.
2. Create a new branch for your feature.
3. Commit your changes.
4. Push to the branch.
5. Create a new Pull Request.

## License
MIT License

---

In [ ]:
%%bash
pip install langchain langchain-experimental openai

In [ ]:
import os
import openai

os.environ['OPENAI_API_KEY'] = "your-api-key"

In [ ]:
%%bash
wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
apt-get install sqlite3

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 768 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.1 [768 kB]
Fetched 768 kB in 0s (1,737 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.1_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.1) ...
Setting up sqlite3 (3.37.2-2ubuntu0.1) ...
Processing triggers for man-db (2.10.2-1) ...


--2023-12-02 19:24:27--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864746 (1.8M) [text/plain]
Saving to: ‘Chinook_Sqlite.sql’

     0K .......... .......... .......... .......... ..........  2% 2.32M 1s
    50K .......... .......... .......... .......... ..........  5% 12.0M 0s
   100K .......... .......... .......... .......... ..........  8% 5.69M 0s
   150K .......... .......... .......... .......... .......... 10% 18.1M 0s
   200K .......... .......... .......... .......... .......... 13% 44.6M 0s
   250K .......... .......... .......... .......... .......... 16% 6.66M 0s
   300K .......... .......... .......... .......... .......... 19

In [ ]:
%%bash
sqlite3 Chinook.db < Chinook_Sqlite.sql

In [ ]:
%%bash
sqlite3 Chinook.db "SELECT * FROM Artist LIMIT 10;"

1|AC/DC
2|Accept
3|Aerosmith
4|Alanis Morissette
5|Alice In Chains
6|Antônio Carlos Jobim
7|Apocalyptica
8|Audioslave
9|BackBeat
10|Billy Cobham


In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


# Connect to the Chinook database
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
import time

try:
    response = db_chain.run("How many employees are there, and who are the employees?")
    print(response)
except Exception as e:
    if "RateLimitError" in str(e):
        print("Rate limit exceeded. Waiting before retrying...")
        time.sleep(60)
        response = db_chain.run("How many employees are there?")
        print(response)
    else:
        raise



> Entering new SQLDatabaseChain chain...
How many employees are there, and who are the employees?
SQLQuery:SELECT COUNT(*), "FirstName", "LastName" FROM "Employee" LIMIT 5;
SQLResult: [(8, 'Andrew', 'Adams')]
Answer:There are 8 employees, including Andrew Adams.
> Finished chain.
There are 8 employees, including Andrew Adams.


In [ ]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI

chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
query = chain.invoke({"question": "what are the names of all the employees"})
print(query)

print(db.run(query))

SELECT "FirstName", "LastName" FROM "Employee"
[('Andrew', 'Adams'), ('Nancy', 'Edwards'), ('Jane', 'Peacock'), ('Margaret', 'Park'), ('Steve', 'Johnson'), ('Michael', 'Mitchell'), ('Robert', 'King'), ('Laura', 'Callahan')]


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

print(agent_executor.run("Describe the playlisttrack table"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: The PlaylistTrack table looks relevant, so I should query the schema of that table.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.

> Finished chain.
The PlaylistTrack table contains